In [ ]:
#!pip install -qU langgraph==0.2.14 langchain==0.2.14 langchain_openai==0.1.23 langchain_core==0.2.35 langchain-community

#!pip install -qU --disable-pip-version-check qdrant-client pymupdf tiktoken

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

##RAG

In [3]:
!mkdir data

In [24]:
from langchain.document_loaders import DirectoryLoader

path ="data/"
loader = DirectoryLoader(path)
docs = loader.load()

In [1]:
# Import our custom PDF section chunker
from pdf_section_chunker import PDFSectionChunker, chunk_northwind_pdf, get_encoding_info, TIKTOKEN_AVAILABLE

# Show available encodings
if TIKTOKEN_AVAILABLE:
    print("🔢 Available Tiktoken Encodings:")
    for encoding, models in get_encoding_info().items():
        print(f"   {encoding}: {models}")
else:
    print("⚠️ Tiktoken not available - using character-based chunking")


🔢 Available Tiktoken Encodings:
   cl100k_base: GPT-4, GPT-3.5-turbo, text-embedding-ada-002
   p50k_base: text-davinci-002, text-davinci-003
   r50k_base: GPT-3 models (davinci, curie, babbage, ada)
   gpt2: GPT-2 models


In [2]:
# Process the Northwind PDF with intelligent section-based chunking
print("🔄 Processing Northwind_Traders_Database_Overview.pdf with section-based chunking...")

# Use our advanced chunker with token optimization for embedding models
chunks = chunk_northwind_pdf(
    pdf_path="data/Northwind_Traders_Database_Overview.pdf",
    use_tokens=True,
    encoding_name="cl100k_base"  # Optimal for OpenAI models
)

print(f"✅ Created {len(chunks)} intelligent chunks from the PDF")

# Display statistics
if chunks:
    char_counts = [chunk.metadata.get('char_count', len(chunk.content)) for chunk in chunks]
    token_counts = [chunk.metadata.get('token_count', 0) for chunk in chunks]
    
    print(f"\n📊 Chunk Statistics:")
    print(f"   Character count - Min: {min(char_counts)}, Max: {max(char_counts)}, Avg: {sum(char_counts)//len(char_counts)}")
    
    if any(token_counts):
        print(f"   Token count - Min: {min(token_counts)}, Max: {max(token_counts)}, Avg: {sum(token_counts)//len(token_counts)}")
        print(f"   Chunking method: {chunks[0].metadata.get('chunking_method', 'unknown')}")
        print(f"   Encoding: {chunks[0].metadata.get('encoding', 'N/A')}")

# Show sample chunks
print(f"\n📋 Sample Chunks:")
for i, chunk in enumerate(chunks[:3]):
    char_count = chunk.metadata.get('char_count', len(chunk.content))
    token_count = chunk.metadata.get('token_count', 'N/A')
    
    print(f"\n   Chunk {i+1}: {chunk.title}")
    print(f"   Page {chunk.page_number} | Level {chunk.section_level} | {char_count} chars | {token_count} tokens")
    preview = chunk.content[:150] + "..." if len(chunk.content) > 150 else chunk.content
    print(f"   Preview: {preview}")


🔄 Processing Northwind_Traders_Database_Overview.pdf with section-based chunking...
✓ Using token-based chunking with cl100k_base encoding
✅ Created 34 intelligent chunks from the PDF

📊 Chunk Statistics:
   Character count - Min: 159, Max: 2049, Avg: 1257
   Token count - Min: 34, Max: 396, Avg: 248
   Chunking method: tokens
   Encoding: cl100k_base

📋 Sample Chunks:

   Chunk 1: Northwind Traders Database Overview
   Page 1 | Level 1 | 427 chars | 77 tokens
   Preview: Northwind Traders is a fictional wholesale food import-export company, and the Northwind database models its day-to-day business operations. This samp...

   Chunk 2: ERP
   Page 1 | Level 4 | 534 chars | 88 tokens
   Preview: (Enterprise Resource Planning) schema for a sales company . The database captures all sales transactions between Northwind and its customers, as well ...

   Chunk 3: Core Business Processes Represented
   Page 1 | Level 1 | 407 chars | 67 tokens
   Preview: Northwind’s schema is designed to sup

In [3]:
# Convert our DocumentChunk objects to LangChain Document format
from langchain.schema import Document

def convert_chunks_to_langchain_docs(chunks):
    """Convert our DocumentChunk objects to LangChain Document format."""
    langchain_docs = []
    
    for chunk in chunks:
        # Create metadata that includes all our enhanced information
        metadata = {
            "source": chunk.metadata.get('source', 'Northwind_Traders_Database_Overview.pdf'),
            "page": chunk.page_number,
            "section_title": chunk.title,
            "section_level": chunk.section_level,
            "char_count": chunk.metadata.get('char_count', len(chunk.content)),
            "token_count": chunk.metadata.get('token_count', 0),
            "chunking_method": chunk.metadata.get('chunking_method', 'unknown'),
            "encoding": chunk.metadata.get('encoding', 'N/A'),
            "chunk_type": chunk.metadata.get('chunk_type', 'section'),
            "is_split": chunk.metadata.get('is_split', False)
        }
        
        # Create LangChain Document
        doc = Document(
            page_content=chunk.content,
            metadata=metadata
        )
        langchain_docs.append(doc)
    
    return langchain_docs

# Convert our smart chunks to LangChain format
smart_docs = convert_chunks_to_langchain_docs(chunks)

print(f"📚 Converted {len(smart_docs)} smart chunks to LangChain Document format")
print(f"📄 Sample metadata: {smart_docs[0].metadata}")

# Replace the old split_chunks with our smart documents
split_chunks = smart_docs


📚 Converted 34 smart chunks to LangChain Document format
📄 Sample metadata: {'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 1, 'section_title': 'Northwind Traders Database Overview', 'section_level': 1, 'char_count': 427, 'token_count': 77, 'chunking_method': 'tokens', 'encoding': 'cl100k_base', 'chunk_type': 'section', 'is_split': False}


In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
# import tiktoken
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# def tiktoken_len(text):
#     tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
#         text,
#     )
#     return len(tokens)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 200,
#     chunk_overlap = 20,
#     length_function = tiktoken_len,
# )

# split_chunks = text_splitter.split_documents(docs)

# from langchain_openai.embeddings import OpenAIEmbeddings

# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

from langchain_community.vectorstores import Qdrant


qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="northwind_data",
)

qdrant_retriever = qdrant_vectorstore.as_retriever()

In [10]:
qdrant_retriever.invoke("What day is it today?")

[Document(metadata={'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 4, 'section_title': 'Orders Table (Part 2)', 'section_level': 3, 'char_count': 435, 'token_count': 101, 'chunking_method': 'tokens', 'encoding': 'cl100k_base', 'chunk_type': 'section', 'is_split': True, '_id': '807d92635838448e8e344c940c894d79', '_collection_name': 'northwind_data'}, page_content='For example, an order record might show that Order #10248 was placed by Customer ALFKI on July 4, 2025 , entered by Employee #5 , and shipped via Speedy Express on July 9, 2025 with a freight charge of $32. Freight and shipper data can be used for logistics analysis (e.g. average shipping times or costs), while the employee and customer links support sales performance tracking (e.g. orders per employee, order history per customer).'),
 Document(metadata={'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 9, 'section_title': 'Scenario 1: Processing a Customer Order (Part 1)', 'section_level': 3

In [11]:
from langchain.schema import BaseRetriever, Document
from typing import List, Any
from pydantic import Field

class ScoreFilteredRetriever(BaseRetriever):
    vectorstore: Any = Field()
    score_threshold: float = Field(default=0.5)
    k: int = Field(default=5)
    
    class Config:
        arbitrary_types_allowed = True
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=self.k)
        
        # Filter by score threshold
        filtered_docs = [
            doc for doc, score in docs_with_scores 
            if score >= self.score_threshold
        ]
        
        return filtered_docs

# Use it
filtered_retriever = ScoreFilteredRetriever(
    vectorstore=qdrant_vectorstore,  # Use keyword arguments
    score_threshold=0.3,
    k=10
)

In [13]:
filtered_retriever.invoke("How many orders are there from the US?")

[Document(metadata={'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 4, 'section_title': 'Orders Table (Part 1)', 'section_level': 3, 'char_count': 1815, 'token_count': 365, 'chunking_method': 'tokens', 'encoding': 'cl100k_base', 'chunk_type': 'section', 'is_split': True, '_id': 'ebfd43b2e6084ddfbe75d9a5ae8c19d9', '_collection_name': 'northwind_data'}, page_content='The Orders table represents sales orders placed by customers. Each row in Orders is a single order, identified by a unique OrderID (primary key). The table captures both who and when for each sale, as well as how it is fulfilled. Key columns include: - CustomerID – which customer placed the order (foreign key to Customers table). - EmployeeID – which employee handled or entered the order (foreign key to Employees table, indicating the sales representative or order taker). - OrderDate – the date the order was placed. - RequiredDate – the date by which the customer requested the goods (often used for scheduli

In [14]:
from langchain_core.prompts import ChatPromptTemplate



RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [15]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

In [53]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | filtered_retriever, "question": itemgetter("question")}
    | rag_prompt | openai_chat_model | StrOutputParser()
)

In [ ]:
# FIXED VERSION: Enhanced retriever with section awareness and token budget management
from langchain.schema import BaseRetriever, Document
from typing import List, Any, Dict
from pydantic import Field

class FixedSectionAwareRetriever(BaseRetriever):
    """Enhanced retriever that considers section hierarchy and manages token budgets."""
    
    vectorstore: Any = Field()
    score_threshold: float = Field(default=0.3)
    k: int = Field(default=10)
    max_tokens: int = Field(default=2000)  # Token budget for context
    prefer_high_level_sections: bool = Field(default=True)
    
    class Config:
        arbitrary_types_allowed = True
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Get initial candidates
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=self.k * 2)
        
        # Filter by score threshold and keep score information
        filtered_docs_with_scores = [
            (doc, score) for doc, score in docs_with_scores 
            if score >= self.score_threshold
        ]
        
        # If preferring high-level sections, sort by section level and score
        if self.prefer_high_level_sections:
            # Sort by section level (lower numbers = higher levels) and score
            filtered_docs_with_scores.sort(key=lambda item: (item[0].metadata.get('section_level', 999), -item[1]))
        
        # Apply token budget management
        selected_docs = []
        total_tokens = 0
        
        for doc, score in filtered_docs_with_scores:
            doc_tokens = doc.metadata.get('token_count', len(doc.page_content) // 4)
            
            if total_tokens + doc_tokens <= self.max_tokens:
                selected_docs.append(doc)
                total_tokens += doc_tokens
            elif len(selected_docs) == 0:  # Ensure we return at least one document
                selected_docs.append(doc)
                break
        
        return selected_docs[:self.k]

# Create FIXED enhanced retriever
enhanced_retriever = FixedSectionAwareRetriever(
    vectorstore=qdrant_vectorstore,
    score_threshold=0.2,  # Lower threshold for more results
    k=8,
    max_tokens=1500,  # Leave room for the question and response
    prefer_high_level_sections=True
)

print("✅ Created FIXED section-aware retriever with token budget management")


In [56]:
rag_chain.invoke({"question" : "Who are the top performing employees?"})

"I don't know. The provided context does not include information about employee performance or any metrics that would identify top performing employees."

In [23]:
# Enhanced retriever with section awareness and token budget management
from langchain.schema import BaseRetriever, Document
from typing import List, Any, Dict
from pydantic import Field

class SectionAwareRetriever(BaseRetriever):
    """Enhanced retriever that considers section hierarchy and manages token budgets."""
    
    vectorstore: Any = Field()
    score_threshold: float = Field(default=0.3)
    k: int = Field(default=10)
    max_tokens: int = Field(default=2000)  # Token budget for context
    prefer_high_level_sections: bool = Field(default=True)
    
    class Config:
        arbitrary_types_allowed = True
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Get initial candidates
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=self.k * 2)
        
        # Filter by score threshold
        filtered_docs = [
            doc for doc, score in docs_with_scores 
            if score >= self.score_threshold
        ]
        
        # If preferring high-level sections, boost their scores
        if self.prefer_high_level_sections:
            # Sort by section level (lower numbers = higher levels) and score
            filtered_docs.sort(key=lambda doc: (doc.metadata.get('section_level', 999), -docs_with_scores[filtered_docs.index(doc)][1]))
        
        # Apply token budget management
        selected_docs = []
        total_tokens = 0
        
        for doc in filtered_docs:
            doc_tokens = doc.metadata.get('token_count', len(doc.page_content) // 4)
            
            if total_tokens + doc_tokens <= self.max_tokens:
                selected_docs.append(doc)
                total_tokens += doc_tokens
            elif len(selected_docs) == 0:  # Ensure we return at least one document
                selected_docs.append(doc)
                break
        
        return selected_docs[:self.k]

# Create enhanced retriever
enhanced_retriever = SectionAwareRetriever(
    vectorstore=qdrant_vectorstore,
    score_threshold=0.2,  # Lower threshold for more results
    k=8,
    max_tokens=1500,  # Leave room for the question and response
    prefer_high_level_sections=True
)

print("✅ Created section-aware retriever with token budget management")


✅ Created section-aware retriever with token budget management


In [24]:
# Enhanced RAG prompt that leverages section information
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser  # Add missing import

ENHANCED_RAG_PROMPT = """
CONTEXT SECTIONS:
{context}

QUERY:
{question}

You are an expert assistant with access to the Northwind Traders Database Overview documentation. 
The context above is organized by document sections to preserve the logical structure of the information.

When answering:
1. Reference specific sections when relevant (e.g., "According to the [Section Name] section...")
2. Maintain the hierarchical structure of information in your response
3. If information spans multiple sections, clearly indicate this
4. Use the section titles to understand the context and scope of each piece of information
5. If you cannot answer the question based on the provided sections, say so clearly

Answer the question using the available context sections:
"""

def format_context_with_sections(docs):
    """Format retrieved documents to show section hierarchy."""
    if not docs:
        return "No relevant sections found."
    
    formatted_sections = []
    for i, doc in enumerate(docs):
        section_title = doc.metadata.get('section_title', f'Section {i+1}')
        section_level = doc.metadata.get('section_level', 0)
        page = doc.metadata.get('page', 'Unknown')
        token_count = doc.metadata.get('token_count', 0)
        
        # Create hierarchical indicator
        level_indicator = "  " * (section_level - 1) + f"Level {section_level}: " if section_level > 0 else ""
        
        section_header = f"[{level_indicator}{section_title} (Page {page}, {token_count} tokens)]"
        formatted_sections.append(f"{section_header}\n{doc.page_content.strip()}")
    
    return "\n\n---\n\n".join(formatted_sections)

# Create enhanced RAG chain
enhanced_rag_prompt = ChatPromptTemplate.from_template(ENHANCED_RAG_PROMPT)

from operator import itemgetter
from langchain_core.runnables import RunnableLambda

enhanced_rag_chain = (
    {
        "context": itemgetter("question") | enhanced_retriever | RunnableLambda(format_context_with_sections),
        "question": itemgetter("question")
    }
    | enhanced_rag_prompt 
    | openai_chat_model 
    | StrOutputParser()
)

print("✅ Created enhanced RAG chain with section-aware formatting")


✅ Created enhanced RAG chain with section-aware formatting


In [25]:
# Test the enhanced RAG system
test_question = "What are the main entities and relationships in the Northwind database?"

print("🤖 Testing Enhanced Section-Aware RAG:")
print(f"Question: {test_question}")
print("\n" + "="*80)

# Get the response
enhanced_response = enhanced_rag_chain.invoke({"question": test_question})
print(enhanced_response)

print("\n" + "="*80)

# Let's also show what sections were retrieved
retrieved_docs = enhanced_retriever.invoke(test_question)
print(f"\n📚 Retrieved {len(retrieved_docs)} sections:")

total_context_tokens = 0
for i, doc in enumerate(retrieved_docs):
    section_title = doc.metadata.get('section_title', f'Section {i+1}')
    page = doc.metadata.get('page', '?')
    level = doc.metadata.get('section_level', 0)
    tokens = doc.metadata.get('token_count', 0)
    total_context_tokens += tokens
    
    print(f"  {i+1}. Level {level}: '{section_title}' (Page {page}, {tokens} tokens)")

print(f"\n🔢 Total context tokens used: {total_context_tokens}")
print(f"💡 Token efficiency: Using only {total_context_tokens} tokens for comprehensive context!")


🤖 Testing Enhanced Section-Aware RAG:
Question: What are the main entities and relationships in the Northwind database?



ValueError: Document(metadata={'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 3, 'section_title': 'Schema Walkthrough: Key Tables and Relationships', 'section_level': 1, 'char_count': 1192, 'token_count': 228, 'chunking_method': 'tokens', 'encoding': 'cl100k_base', 'chunk_type': 'section', 'is_split': False, '_id': '4712e9c8418f495eb4709d5c40692b84', '_collection_name': 'northwind_data'}, page_content='The Northwind database is a relational schema with multiple tables, each representing an entity in the business. The design is highly normalized – information is broken into logical tables with relationships (via primary and foreign keys) connecting them. Below, we walk through each of the key tables, explaining their purpose, structure, and how they interrelate. Figure: Entity-Relationship Diagram (ERD) of the Northwind database schema, showing the key tables and their relationships. Each box is a table (with major columns listed), and lines indicate foreign key relat

In [21]:
# Test with different types of questions to showcase the system's capabilities

test_questions = [
    "Who are the top performing employees in Northwind?",
    "What products does Northwind sell and how are they categorized?",
    "Describe the customer base and their geographic distribution",
    "How does the order processing system work?",
    "What are the key business relationships in the Northwind database model?"
]

print("🧪 Testing Enhanced RAG with Various Questions:")
print("=" * 100)

for i, question in enumerate(test_questions, 1):
    print(f"\n🔍 Question {i}: {question}")
    print("-" * 60)
    
    # Get response
    response = enhanced_rag_chain.invoke({"question": question})
    print(response)
    
    # Show retrieved sections
    docs = enhanced_retriever.invoke(question)
    section_info = [f"{doc.metadata.get('section_title', 'Unknown')} (Level {doc.metadata.get('section_level', 0)})" 
                   for doc in docs]
    tokens_used = sum(doc.metadata.get('token_count', 0) for doc in docs)
    
    print(f"\n📊 Retrieved sections: {', '.join(section_info[:3])}{'...' if len(section_info) > 3 else ''}")
    print(f"🔢 Context tokens: {tokens_used}")
    
    if i < len(test_questions):
        print("\n" + "=" * 100)


🧪 Testing Enhanced RAG with Various Questions:

🔍 Question 1: Who are the top performing employees in Northwind?
------------------------------------------------------------


ValueError: Document(metadata={'source': 'data/Northwind_Traders_Database_Overview.pdf', 'page': 7, 'section_title': 'Employees Table (Part 2)', 'section_level': 3, 'char_count': 643, 'token_count': 120, 'chunking_method': 'tokens', 'encoding': 'cl100k_base', 'chunk_type': 'section', 'is_split': True, '_id': '65d810e4a688475ea5bec93baf2f9ebe', '_collection_name': 'northwind_data'}, page_content='By querying this, one can tally each employee’s sales or see which customers each employee has dealt with. The Employees table combined with territory assignments (described next) also allows Northwind to map its workforce to sales regions. Additionally, from an analytics or HR perspective, having all these details enables the company to track employee performance (sales figures by employee via Orders linkage), tenure (via HireDate), and other metrics. In summary, the Employees table defines who works at Northwind, what their role is, and connects them both to the organizational hierarchy and t

In [72]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser  # Add missing import

RAG_PROMPT_SUMMARY = """
CONTEXT:
{context}

ORIGINAL QUERY:
{question}

You are a document summarizer. Analyze the provided context and create a summary that addresses the query as best as possible.

REQUIREMENTS:
- If the context contains information about customers, orders, or sales data, extract and summarize it
- Even if the exact query cannot be answered, summarize any related information found
- If you find customer data but not specifically "top customers", still summarize what customer information is available
- Keep the summary under 200 words
- Only return "No relevant information found" if the context is completely unrelated to the query topic

SUMMARY:
"""

rag_prompt_summary = ChatPromptTemplate.from_template(RAG_PROMPT_SUMMARY)

# rag_chain_summary = (
#     {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
#     | rag_prompt_summary | openai_chat_model | StrOutputParser()
# )

from operator import itemgetter
from langchain_core.runnables import RunnableLambda

def format_and_filter_docs(docs):
    """Format docs and add metadata for better summarization"""
    if not docs:
        return "No documents retrieved."
    
    formatted_context = []
    for i, doc in enumerate(docs):
        # Add source info if available
        source = doc.metadata.get('source', f'Document {i+1}')
        content = doc.page_content.strip()
        formatted_context.append(f"[{source}]: {content}")
    
    return "\n\n".join(formatted_context)

# Your summary chain
rag_chain_summary = (
    {
        "context": itemgetter("question") | qdrant_retriever | RunnableLambda(format_and_filter_docs),
        "question": itemgetter("question")
    }
    | rag_prompt_summary 
    | openai_chat_model 
    | StrOutputParser()
)

In [73]:
rag_chain_summary.invoke({"question" : "Who are the top customers of Northwind in the US?"})

'The provided context describes the Northwind sample database, which contains 14 interrelated tables relevant to a fictitious trading business, including data on orders, customers, and products. Specifically, the Orders table captures sales orders placed by customers, indicating a potential source for identifying customer activity and sales. However, the context does not include specific data or metrics regarding the top customers in the US or any sales figures that could be leveraged to ascertain customer rankings. Additionally, it references the CustomerDemographics table without providing details about its content. Overall, while the Northwind database features comprehensive customer and order information that could lead to identifying top customers, the exact details of those customers are not presented in the context provided.'

In [70]:
docs_with_scores = qdrant_vectorstore.similarity_search_with_score(
    "Who are the top customers of Northwind in the US?", k=5
)

print("Similarity scores:")
for doc, score in docs_with_scores:
    print(f"Score: {score:.3f}")
    print(f"Content: {doc.page_content[:200]}...")
    print("---")

Similarity scores:
Score: 0.579
Content: 42

43

44

45

46

47

48

49

Northwind Database

https://techwriter.me/downloads/samples/Database/Access2003Northwind.pdf

30

51

52

53

54

55

58

59

60

raw.githubusercontent.com

https://raw...
---
Score: 0.548
Content: Northwind Database Schema

Overview: The Northwind sample database contains 14 interrelated tables representing a fictitious trading

business. The schema includes tables for orders, customers, employ...
---
Score: 0.485
Content: 17

. Similarly, each Product can appear in many

Order_Details records (one-to-many from Products to Order_Details)

18

. Through Order_Details,

Orders and Products are related in a many-to-many ma...
---
Score: 0.470
Content: Foreign Keys: (None) – US_States is a standalone reference table.



Relationships: This table is not linked via foreign keys to the rest of the Northwind schema. It may

be used for address data cons...
---
Score: 0.464
Content: Foreign Keys: (None) – the Customers ta

In [106]:
import pandas as pd
import psycopg2
from typing import List
from datetime import datetime

def create_comprehensive_northwind_business_documents(
    host: str,
    username: str,
    password: str,
    database: str = "neondb",
    port: int = 5432,
    schema: str = "northwind"
) -> List[str]:
    """
    Execute comprehensive SQL queries against PostgreSQL Northwind database hosted on Neon 
    and create detailed business-friendly documents for vector search and RAG applications.
    
    This expanded version includes deep analysis of all business aspects including:
    - Customer demographics and behavior patterns
    - Product performance and inventory management
    - Employee productivity and territory analysis
    - Supplier relationships and logistics
    - Financial performance and trends
    - Geographic distribution and shipping patterns
    """
    
    # Create connection string for Neon
    conn_string = f"postgresql://{username}:{password}@{host}:{port}/{database}?sslmode=require"
    
    documents = []
    
    try:
        print("Connecting to Northwind database and generating comprehensive business documents...")
        
        # 1. COMPREHENSIVE CUSTOMER ANALYSIS
        print("Generating customer analysis document...")
        
        # Customer demographics and distribution
        customer_demographics_query = f"""
        SELECT 
            country,
            city,
            COUNT(*) as customer_count,
            STRING_AGG(DISTINCT contact_title, ', ') as job_titles,
            STRING_AGG(company_name, '; ' ORDER BY company_name) as sample_companies
        FROM {schema}.customers 
        GROUP BY country, city
        ORDER BY customer_count DESC, country, city
        """
        
        df_demographics = pd.read_sql_query(customer_demographics_query, conn_string)
        
        customer_doc = "NORTHWIND COMPREHENSIVE CUSTOMER ANALYSIS:\n\n"
        customer_doc += f"CUSTOMER BASE OVERVIEW:\n"
        customer_doc += f"Northwind serves {df_demographics['customer_count'].sum()} customers across {len(df_demographics['country'].unique())} countries and {len(df_demographics)} cities.\n\n"
        
        # Country analysis
        country_summary = df_demographics.groupby('country').agg({
            'customer_count': 'sum',
            'city': 'count'
        }).sort_values('customer_count', ascending=False)
        
        customer_doc += "CUSTOMER DISTRIBUTION BY COUNTRY:\n"
        for country, row in country_summary.head(15).iterrows():
            customer_doc += f"- {country}: {row['customer_count']} customers across {row['city']} cities\n"
        
        # Detailed customer profiles with contact information
        detailed_customers_query = f"""
        SELECT 
            customer_id,
            company_name,
            contact_name,
            contact_title,
            city,
            region,
            country,
            phone,
            fax
        FROM {schema}.customers
        ORDER BY country, city, company_name
        """
        
        df_detailed = pd.read_sql_query(detailed_customers_query, conn_string)
        
        customer_doc += "\n\nDETAILED CUSTOMER DIRECTORY:\n"
        
        # Group by country for better organization
        for country in df_detailed['country'].unique()[:10]:  # Top 10 countries
            country_customers = df_detailed[df_detailed['country'] == country]
            customer_doc += f"\n{country.upper()} ({len(country_customers)} customers):\n"
            
            for _, customer in country_customers.head(8).iterrows():  # Top 8 per country
                region_info = f", {customer['region']}" if pd.notna(customer['region']) else ""
                fax_info = f", Fax: {customer['fax']}" if pd.notna(customer['fax']) else ""
                customer_doc += f"  • {customer['company_name']} - {customer['contact_name']} ({customer['contact_title']})\n"
                customer_doc += f"    Location: {customer['city']}{region_info}, Phone: {customer['phone']}{fax_info}\n"
        
        documents.append(customer_doc)
        
        # 2. CUSTOMER PURCHASING BEHAVIOR AND TOP PERFORMERS
        print("Generating customer purchasing behavior analysis...")
        
        customer_behavior_query = f"""
        WITH customer_metrics AS (
            SELECT 
                c.customer_id,
                c.company_name,
                c.contact_name,
                c.city,
                c.region,
                c.country,
                COUNT(DISTINCT o.order_id) as total_orders,
                COUNT(DISTINCT DATE_PART('year', o.order_date)) as years_active,
                MIN(o.order_date) as first_order_date,
                MAX(o.order_date) as last_order_date,
                ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as total_revenue,
                ROUND(AVG(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as avg_order_value,
                SUM(od.quantity) as total_items_purchased,
                ROUND(AVG(o.freight)::numeric, 2) as avg_shipping_cost,
                COUNT(DISTINCT od.product_id) as unique_products_bought
            FROM {schema}.customers c
            JOIN {schema}.orders o ON c.customer_id = o.customer_id
            JOIN {schema}.order_details od ON o.order_id = od.order_id
            GROUP BY c.customer_id, c.company_name, c.contact_name, c.city, c.region, c.country
        )
        SELECT * FROM customer_metrics
        ORDER BY total_revenue DESC
        """
        
        df_behavior = pd.read_sql_query(customer_behavior_query, conn_string)
        
        behavior_doc = "NORTHWIND CUSTOMER PURCHASING BEHAVIOR ANALYSIS:\n\n"
        
        # Top customers by revenue
        behavior_doc += "TOP 20 CUSTOMERS BY TOTAL REVENUE:\n"
        for i, row in df_behavior.head(20).iterrows():
            region = f", {row['region']}" if pd.notna(row['region']) else ""
            years_span = f"{row['first_order_date']:.10}" + " to " + f"{row['last_order_date']:.10}"
            
            behavior_doc += f"{i+1}. {row['company_name']} ({row['country']})\n"
            behavior_doc += f"   • Total Revenue: ${row['total_revenue']:,.2f} across {row['total_orders']} orders\n"
            behavior_doc += f"   • Average Order Value: ${row['avg_order_value']:,.2f}\n"
            behavior_doc += f"   • Active Period: {years_span} ({row['years_active']} years)\n"
            behavior_doc += f"   • Location: {row['city']}{region}\n"
            behavior_doc += f"   • Products Diversity: {row['unique_products_bought']} different products, {row['total_items_purchased']} total items\n\n"
        
        # Customer segmentation analysis
        behavior_doc += "CUSTOMER SEGMENTATION ANALYSIS:\n"
        revenue_percentiles = df_behavior['total_revenue'].quantile([0.25, 0.5, 0.75, 0.9, 0.95])
        
        behavior_doc += f"• Premium Customers (Top 5%): ${revenue_percentiles[0.95]:,.2f}+ revenue ({len(df_behavior[df_behavior['total_revenue'] >= revenue_percentiles[0.95]])} customers)\n"
        behavior_doc += f"• High-Value Customers (Top 10%): ${revenue_percentiles[0.9]:,.2f}+ revenue ({len(df_behavior[df_behavior['total_revenue'] >= revenue_percentiles[0.9]])} customers)\n"
        behavior_doc += f"• Regular Customers (Median): ${revenue_percentiles[0.5]:,.2f} revenue\n"
        behavior_doc += f"• Average Order Value Range: ${df_behavior['avg_order_value'].min():,.2f} - ${df_behavior['avg_order_value'].max():,.2f}\n"
        
        # Geographic revenue distribution
        geographic_revenue = df_behavior.groupby('country').agg({
            'total_revenue': 'sum',
            'total_orders': 'sum',
            'company_name': 'count'
        }).sort_values('total_revenue', ascending=False)
        
        behavior_doc += "\nREVENUE BY COUNTRY:\n"
        for country, row in geographic_revenue.head(10).iterrows():
            avg_revenue_per_customer = row['total_revenue'] / row['company_name']
            behavior_doc += f"• {country}: ${row['total_revenue']:,.2f} total (${avg_revenue_per_customer:,.2f} avg per customer)\n"
        
        documents.append(behavior_doc)
        
        # 3. COMPREHENSIVE PRODUCT CATALOG AND PERFORMANCE
        print("Generating comprehensive product analysis...")
        
        product_analysis_query = f"""
        WITH product_performance AS (
            SELECT 
                p.product_id,
                p.product_name,
                c.category_name,
                s.company_name as supplier_name,
                s.country as supplier_country,
                p.quantity_per_unit,
                p.unit_price,
                p.units_in_stock,
                p.units_on_order,
                p.reorder_level,
                p.discontinued,
                COALESCE(SUM(od.quantity), 0) as total_quantity_sold,
                COALESCE(ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2), 0) as total_revenue,
                COALESCE(COUNT(DISTINCT od.order_id), 0) as orders_count,
                COALESCE(ROUND(AVG(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2), 0) as avg_order_line_value
            FROM {schema}.products p
            JOIN {schema}.categories c ON p.category_id = c.category_id
            JOIN {schema}.suppliers s ON p.supplier_id = s.supplier_id
            LEFT JOIN {schema}.order_details od ON p.product_id = od.product_id
            GROUP BY p.product_id, p.product_name, c.category_name, s.company_name, s.country,
                     p.quantity_per_unit, p.unit_price, p.units_in_stock, p.units_on_order, 
                     p.reorder_level, p.discontinued
        )
        SELECT * FROM product_performance
        ORDER BY total_revenue DESC
        """
        
        df_products = pd.read_sql_query(product_analysis_query, conn_string)
        
        product_doc = "NORTHWIND COMPREHENSIVE PRODUCT CATALOG AND PERFORMANCE:\n\n"
        
        # Category overview
        category_performance = df_products.groupby('category_name').agg({
            'product_id': 'count',
            'total_revenue': 'sum',
            'total_quantity_sold': 'sum',
            'unit_price': 'mean',
            'units_in_stock': 'sum'
        }).sort_values('total_revenue', ascending=False)
        
        product_doc += f"PRODUCT PORTFOLIO OVERVIEW:\n"
        product_doc += f"Total Products: {len(df_products)} across {len(category_performance)} categories\n"
        product_doc += f"Active Products: {len(df_products[df_products['discontinued'] == 0])}\n"
        product_doc += f"Discontinued Products: {len(df_products[df_products['discontinued'] == 1])}\n\n"
        
        product_doc += "CATEGORY PERFORMANCE ANALYSIS:\n"
        for category, row in category_performance.iterrows():
            product_doc += f"• {category}: {row['product_id']} products, ${row['total_revenue']:,.2f} revenue\n"
            product_doc += f"  - {row['total_quantity_sold']:,.0f} units sold, avg price: ${row['unit_price']:,.2f}\n"
            product_doc += f"  - Current inventory: {row['units_in_stock']:,.0f} units\n"
        
        # Top performing products
        product_doc += "\nTOP 25 PRODUCTS BY REVENUE:\n"
        for i, row in df_products.head(25).iterrows():
            discontinued_status = " [DISCONTINUED]" if row['discontinued'] == 1 else ""
            stock_status = "⚠️ LOW STOCK" if row['units_in_stock'] <= row['reorder_level'] else "✅ IN STOCK"
            
            product_doc += f"{i+1}. {row['product_name']} ({row['category_name']}){discontinued_status}\n"
            product_doc += f"   • Revenue: ${row['total_revenue']:,.2f} from {row['total_quantity_sold']} units sold\n"
            product_doc += f"   • Price: ${row['unit_price']:.2f} per {row['quantity_per_unit']}\n"
            product_doc += f"   • Supplier: {row['supplier_name']} ({row['supplier_country']})\n"
            product_doc += f"   • Inventory: {row['units_in_stock']} in stock, {row['units_on_order']} on order ({stock_status})\n\n"
        
        # Inventory management insights
        low_stock_products = df_products[df_products['units_in_stock'] <= df_products['reorder_level']]
        high_revenue_low_stock = low_stock_products[low_stock_products['total_revenue'] > df_products['total_revenue'].median()]
        
        product_doc += f"INVENTORY MANAGEMENT ALERTS:\n"
        product_doc += f"• Products below reorder level: {len(low_stock_products)}\n"
        product_doc += f"• High-revenue products with low stock: {len(high_revenue_low_stock)}\n"
        
        if len(high_revenue_low_stock) > 0:
            product_doc += "  Critical reorder needed for:\n"
            for _, product in high_revenue_low_stock.head(5).iterrows():
                product_doc += f"    - {product['product_name']}: {product['units_in_stock']} units (${product['total_revenue']:,.0f} revenue)\n"
        
        documents.append(product_doc)
        
        # 4. SUPPLIER RELATIONSHIPS AND LOGISTICS
        print("Generating supplier analysis...")
        
        supplier_analysis_query = f"""
        WITH supplier_metrics AS (
            SELECT 
                s.supplier_id,
                s.company_name,
                s.contact_name,
                s.contact_title,
                s.city,
                s.region,
                s.country,
                s.phone,
                s.fax,
                COUNT(p.product_id) as products_supplied,
                COUNT(CASE WHEN p.discontinued = 0 THEN 1 END) as active_products,
                ROUND(AVG(p.unit_price)::numeric, 2) as avg_product_price,
                SUM(p.units_in_stock) as total_inventory_units,
                COALESCE(SUM(od.quantity), 0) as total_units_sold,
                COALESCE(ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2), 0) as total_revenue_generated
            FROM {schema}.suppliers s
            JOIN {schema}.products p ON s.supplier_id = p.supplier_id
            LEFT JOIN {schema}.order_details od ON p.product_id = od.product_id
            GROUP BY s.supplier_id, s.company_name, s.contact_name, s.contact_title,
                     s.city, s.region, s.country, s.phone, s.fax
        )
        SELECT * FROM supplier_metrics
        ORDER BY total_revenue_generated DESC
        """
        
        df_suppliers = pd.read_sql_query(supplier_analysis_query, conn_string)
        
        supplier_doc = "NORTHWIND SUPPLIER RELATIONSHIP AND LOGISTICS ANALYSIS:\n\n"
        
        # Supplier overview
        supplier_doc += f"SUPPLIER NETWORK OVERVIEW:\n"
        supplier_doc += f"Total Suppliers: {len(df_suppliers)}\n"
        supplier_doc += f"Geographic Distribution: {len(df_suppliers['country'].unique())} countries\n"
        supplier_doc += f"Total Products Supplied: {df_suppliers['products_supplied'].sum()}\n"
        supplier_doc += f"Active Products: {df_suppliers['active_products'].sum()}\n\n"
        
        # Supplier performance ranking
        supplier_doc += "TOP SUPPLIERS BY REVENUE GENERATION:\n"
        for i, row in df_suppliers.head(15).iterrows():
            region_info = f", {row['region']}" if pd.notna(row['region']) else ""
            fax_info = f", Fax: {row['fax']}" if pd.notna(row['fax']) else ""
            
            supplier_doc += f"{i+1}. {row['company_name']} ({row['country']})\n"
            supplier_doc += f"   • Contact: {row['contact_name']} ({row['contact_title']})\n"
            supplier_doc += f"   • Location: {row['city']}{region_info}\n"
            supplier_doc += f"   • Phone: {row['phone']}{fax_info}\n"
            supplier_doc += f"   • Products: {row['products_supplied']} total ({row['active_products']} active)\n"
            supplier_doc += f"   • Revenue Generated: ${row['total_revenue_generated']:,.2f}\n"
            supplier_doc += f"   • Units Sold: {row['total_units_sold']:,.0f}, Avg Product Price: ${row['avg_product_price']:,.2f}\n\n"
        
        # Geographic supplier distribution
        supplier_by_country = df_suppliers.groupby('country').agg({
            'supplier_id': 'count',
            'products_supplied': 'sum',
            'total_revenue_generated': 'sum'
        }).sort_values('total_revenue_generated', ascending=False)
        
        supplier_doc += "SUPPLIER GEOGRAPHIC DISTRIBUTION:\n"
        for country, row in supplier_by_country.iterrows():
            avg_revenue_per_supplier = row['total_revenue_generated'] / row['supplier_id']
            supplier_doc += f"• {country}: {row['supplier_id']} suppliers, {row['products_supplied']} products, ${row['total_revenue_generated']:,.2f} revenue\n"
            supplier_doc += f"  Average revenue per supplier: ${avg_revenue_per_supplier:,.2f}\n"
        
        documents.append(supplier_doc)
        
        # 5. EMPLOYEE PERFORMANCE AND TERRITORY ANALYSIS
        print("Generating employee and territory analysis...")
        
        employee_analysis_query = f"""
        WITH employee_performance AS (
            SELECT 
                e.employee_id,
                e.first_name || ' ' || e.last_name as full_name,
                e.title,
                e.title_of_courtesy,
                e.birth_date,
                e.hire_date,
                e.city,
                e.region,
                e.country,
                e.home_phone,
                e.reports_to,
                mgr.first_name || ' ' || mgr.last_name as manager_name,
                COUNT(DISTINCT o.order_id) as orders_handled,
                COUNT(DISTINCT o.customer_id) as customers_served,
                COALESCE(ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2), 0) as total_sales,
                COALESCE(ROUND(AVG(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2), 0) as avg_order_value,
                COALESCE(SUM(od.quantity), 0) as total_units_sold,
                COUNT(DISTINCT DATE_PART('year', o.order_date)) as years_active,
                MIN(o.order_date) as first_sale_date,
                MAX(o.order_date) as last_sale_date
            FROM {schema}.employees e
            LEFT JOIN {schema}.employees mgr ON e.reports_to = mgr.employee_id
            LEFT JOIN {schema}.orders o ON e.employee_id = o.employee_id
            LEFT JOIN {schema}.order_details od ON o.order_id = od.order_id
            GROUP BY e.employee_id, e.first_name, e.last_name, e.title, e.title_of_courtesy,
                     e.birth_date, e.hire_date, e.city, e.region, e.country, e.home_phone,
                     e.reports_to, mgr.first_name, mgr.last_name
        )
        SELECT * FROM employee_performance
        ORDER BY total_sales DESC
        """
        
        df_employees = pd.read_sql_query(employee_analysis_query, conn_string)
        
        employee_doc = "NORTHWIND EMPLOYEE PERFORMANCE AND ORGANIZATIONAL ANALYSIS:\n\n"
        
        # Organizational overview
        employee_doc += f"ORGANIZATIONAL STRUCTURE:\n"
        employee_doc += f"Total Employees: {len(df_employees)}\n"
        
        # Management hierarchy
        managers = df_employees[df_employees['reports_to'].isna()]
        subordinates = df_employees[df_employees['reports_to'].notna()]
        
        employee_doc += f"Management Level: {len(managers)} managers\n"
        employee_doc += f"Staff Level: {len(subordinates)} employees\n\n"
        
        # Employee performance ranking
        employee_doc += "EMPLOYEE SALES PERFORMANCE RANKING:\n"
        for i, row in df_employees.iterrows():
            manager_info = f" (Reports to: {row['manager_name']})" if pd.notna(row['manager_name']) else " (Senior Management)"
            years_service = datetime.now().year - pd.to_datetime(row['hire_date']).year if pd.notna(row['hire_date']) else 0
            age = datetime.now().year - pd.to_datetime(row['birth_date']).year if pd.notna(row['birth_date']) else 0
            
            employee_doc += f"{i+1}. {row['full_name']} - {row['title']}\n"
            employee_doc += f"   • Total Sales: ${row['total_sales']:,.2f} across {row['orders_handled']} orders\n"
            employee_doc += f"   • Customers Served: {row['customers_served']}, Avg Order Value: ${row['avg_order_value']:,.2f}\n"
            employee_doc += f"   • Service Period: {years_service} years (hired {str(row['hire_date'])[:10]})\n"
            employee_doc += f"   • Age: {age}, Location: {row['city']}, {row['country']}\n"
            employee_doc += f"   • Contact: {row['home_phone']}{manager_info}\n\n"
        
        # Performance metrics analysis
        total_company_sales = df_employees['total_sales'].sum()
        top_performer = df_employees.iloc[0]
        
        employee_doc += "PERFORMANCE INSIGHTS:\n"
        employee_doc += f"• Top Performer: {top_performer['full_name']} (${top_performer['total_sales']:,.2f} - {(top_performer['total_sales']/total_company_sales*100):.1f}% of total sales)\n"
        employee_doc += f"• Average Sales per Employee: ${df_employees['total_sales'].mean():,.2f}\n"
        employee_doc += f"• Sales Performance Range: ${df_employees['total_sales'].min():,.2f} - ${df_employees['total_sales'].max():,.2f}\n"
        employee_doc += f"• Average Customer Base per Employee: {df_employees['customers_served'].mean():.1f} customers\n"
        
        documents.append(employee_doc)
        
        # 6. SHIPPING AND LOGISTICS ANALYSIS
        print("Generating shipping and logistics analysis...")
        
        shipping_analysis_query = f"""
        WITH shipping_metrics AS (
    SELECT 
        sh.shipper_id,
        sh.company_name as shipper_name,
        sh.phone as shipper_phone,
        COUNT(o.order_id) as total_shipments,
        COUNT(DISTINCT o.customer_id) as customers_served,
        COUNT(DISTINCT o.ship_country) as countries_served,
        ROUND(AVG(o.freight)::numeric, 2) as avg_freight_cost,
        ROUND(SUM(o.freight)::numeric, 2) as total_freight_revenue,
        ROUND(AVG((o.shipped_date::date - o.order_date::date))::numeric, 1) as avg_delivery_days,
        COUNT(CASE WHEN o.shipped_date::date > o.required_date::date THEN 1 END) as late_deliveries,
        ROUND((COUNT(CASE WHEN o.shipped_date::date > o.required_date::date THEN 1 END) * 100.0 / COUNT(o.order_id))::numeric, 2) as late_delivery_rate
    FROM {schema}.shippers sh
    LEFT JOIN {schema}.orders o ON sh.shipper_id = o.ship_via
    WHERE o.shipped_date IS NOT NULL AND o.order_date IS NOT NULL
    GROUP BY sh.shipper_id, sh.company_name, sh.phone
),
route_analysis AS (
    SELECT 
        o.ship_country,
        o.ship_city,
        COUNT(o.order_id) as shipment_count,
        ROUND(AVG(o.freight)::numeric, 2) as avg_freight_cost,
        COUNT(DISTINCT o.customer_id) as customers_in_location
    FROM {schema}.orders o
    WHERE o.shipped_date IS NOT NULL
    GROUP BY o.ship_country, o.ship_city
)
SELECT 
    sm.*,
    (SELECT COUNT(*) FROM route_analysis) as total_shipping_locations
FROM shipping_metrics sm
ORDER BY sm.total_freight_revenue DESC;
        """
        
        df_shipping = pd.read_sql_query(shipping_analysis_query, conn_string)
        
        # Route analysis
        route_query = f"""
        SELECT 
            ship_country,
            ship_city,
            COUNT(order_id) as shipment_count,
            ROUND(AVG(freight)::numeric, 2) as avg_freight_cost,
            COUNT(DISTINCT customer_id) as customers_in_location
        FROM {schema}.orders
        WHERE shipped_date IS NOT NULL
        GROUP BY ship_country, ship_city
        ORDER BY shipment_count DESC
        """
        
        df_routes = pd.read_sql_query(route_query, conn_string)
        
        shipping_doc = "NORTHWIND SHIPPING AND LOGISTICS PERFORMANCE ANALYSIS:\n\n"
        
        # Shipping company performance
        shipping_doc += "SHIPPING PARTNER PERFORMANCE:\n"
        for _, row in df_shipping.iterrows():
            on_time_rate = 100 - row['late_delivery_rate']
            
            shipping_doc += f"• {row['shipper_name']} (Phone: {row['shipper_phone']})\n"
            shipping_doc += f"  - Total Shipments: {row['total_shipments']:,} to {row['customers_served']} customers\n"
            shipping_doc += f"  - Coverage: {row['countries_served']} countries\n"
            shipping_doc += f"  - Freight Revenue: ${row['total_freight_revenue']:,.2f} (Avg: ${row['avg_freight_cost']:.2f} per shipment)\n"
            shipping_doc += f"  - Delivery Performance: {row['avg_delivery_days']:.1f} days average, {on_time_rate:.1f}% on-time rate\n"
            shipping_doc += f"  - Late Deliveries: {row['late_deliveries']} ({row['late_delivery_rate']:.1f}%)\n\n"
        
        # Geographic shipping analysis
        shipping_doc += "TOP SHIPPING DESTINATIONS:\n"
        country_routes = df_routes.groupby('ship_country').agg({
            'shipment_count': 'sum',
            'avg_freight_cost': 'mean',
            'customers_in_location': 'sum',
            'ship_city': 'count'
        }).sort_values('shipment_count', ascending=False)
        
        for country, row in country_routes.head(15).iterrows():
            shipping_doc += f"• {country}: {row['shipment_count']} shipments to {row['ship_city']} cities\n"
            shipping_doc += f"  - {row['customers_in_location']} customers, avg freight: ${row['avg_freight_cost']:.2f}\n"
        
        shipping_doc += "\nTOP CITY DESTINATIONS:\n"
        for _, row in df_routes.head(20).iterrows():
            shipping_doc += f"• {row['ship_city']}, {row['ship_country']}: {row['shipment_count']} shipments\n"
            shipping_doc += f"  - {row['customers_in_location']} customers, avg freight: ${row['avg_freight_cost']:.2f}\n"
        
        documents.append(shipping_doc)
        
        # 7. FINANCIAL PERFORMANCE AND TRENDS ANALYSIS
        print("Generating comprehensive financial analysis...")
        
        financial_analysis_query = f"""
        WITH monthly_performance AS (
            SELECT 
                EXTRACT(year FROM o.order_date) as year,
                EXTRACT(month FROM o.order_date) as month,
                COUNT(DISTINCT o.order_id) as order_count,
                COUNT(DISTINCT o.customer_id) as active_customers,
                COUNT(DISTINCT od.product_id) as products_sold,
                SUM(od.quantity) as units_sold,
                ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as revenue,
                ROUND(AVG(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as avg_order_line_value,
                ROUND(SUM(o.freight)::numeric, 2) as total_freight,
                ROUND(SUM(od.unit_price * od.quantity * od.discount)::numeric, 2) as total_discounts,
                ROUND(AVG(od.discount * 100)::numeric, 2) as avg_discount_percentage
            FROM {schema}.orders o
            JOIN {schema}.order_details od ON o.order_id = od.order_id
            WHERE o.order_date IS NOT NULL
            GROUP BY EXTRACT(year FROM o.order_date), EXTRACT(month FROM o.order_date)
        ),
        quarterly_performance AS (
            SELECT 
                year,
                CASE 
                    WHEN month IN (1,2,3) THEN 'Q1'
                    WHEN month IN (4,5,6) THEN 'Q2'
                    WHEN month IN (7,8,9) THEN 'Q3'
                    ELSE 'Q4'
                END as quarter,
                SUM(order_count) as orders,
                SUM(revenue) as quarterly_revenue,
                AVG(active_customers) as avg_monthly_customers,
                SUM(units_sold) as total_units,
                SUM(total_freight) as freight_revenue
            FROM monthly_performance
            GROUP BY year, CASE 
                WHEN month IN (1,2,3) THEN 'Q1'
                WHEN month IN (4,5,6) THEN 'Q2'
                WHEN month IN (7,8,9) THEN 'Q3'
                ELSE 'Q4'
            END
        )
        SELECT * FROM monthly_performance
        ORDER BY year, month
        """
        
        df_financial = pd.read_sql_query(financial_analysis_query, conn_string)
        
        financial_doc = "NORTHWIND COMPREHENSIVE FINANCIAL PERFORMANCE ANALYSIS:\n\n"
        
        # Overall financial summary
        total_revenue = df_financial['revenue'].sum()
        total_orders = df_financial['order_count'].sum()
        total_discounts = df_financial['total_discounts'].sum()
        
        financial_doc += f"FINANCIAL OVERVIEW:\n"
        financial_doc += f"• Total Revenue: ${total_revenue:,.2f}\n"
        financial_doc += f"• Total Orders: {total_orders:,}\n"
        financial_doc += f"• Total Discounts Given: ${total_discounts:,.2f} ({(total_discounts/total_revenue*100):.1f}% of revenue)\n"
        financial_doc += f"• Average Order Value: ${(total_revenue/total_orders):,.2f}\n"
        financial_doc += f"• Total Units Sold: {df_financial['units_sold'].sum():,}\n\n"
        
        # Yearly performance
        yearly_summary = df_financial.groupby('year').agg({
            'revenue': 'sum',
            'order_count': 'sum',
            'active_customers': 'mean',
            'units_sold': 'sum',
            'total_freight': 'sum',
            'total_discounts': 'sum'
        }).round(2)
        
        financial_doc += "ANNUAL PERFORMANCE BREAKDOWN:\n"
        for year, row in yearly_summary.iterrows():
            year_growth = ""
            if year > yearly_summary.index.min():
                prev_year_revenue = yearly_summary.loc[year-1, 'revenue']
                growth_rate = ((row['revenue'] - prev_year_revenue) / prev_year_revenue * 100)
                year_growth = f" ({growth_rate:+.1f}% vs prior year)"
            
            financial_doc += f"• {int(year)}: ${row['revenue']:,.2f} revenue{year_growth}\n"
            financial_doc += f"  - {int(row['order_count']):,} orders from {row['active_customers']:.0f} avg monthly customers\n"
            financial_doc += f"  - {int(row['units_sold']):,} units sold, ${row['total_freight']:,.2f} freight revenue\n"
            financial_doc += f"  - ${row['total_discounts']:,.2f} in discounts ({(row['total_discounts']/row['revenue']*100):.1f}%)\n"
        
        # Monthly trends analysis
        financial_doc += "\nMONTHLY PERFORMANCE TRENDS:\n"
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        monthly_avg = df_financial.groupby('month').agg({
            'revenue': 'mean',
            'order_count': 'mean',
            'active_customers': 'mean'
        }).round(2)
        
        # Find best and worst performing months
        best_month = monthly_avg['revenue'].idxmax()
        worst_month = monthly_avg['revenue'].idxmin()
        
        financial_doc += f"• Best Month: {month_names[int(best_month)-1]} (${monthly_avg.loc[best_month, 'revenue']:,.2f} avg revenue)\n"
        financial_doc += f"• Weakest Month: {month_names[int(worst_month)-1]} (${monthly_avg.loc[worst_month, 'revenue']:,.2f} avg revenue)\n"
        financial_doc += f"• Seasonal Variance: {((monthly_avg['revenue'].max() - monthly_avg['revenue'].min()) / monthly_avg['revenue'].mean() * 100):.1f}%\n\n"
        
        financial_doc += "MONTHLY AVERAGE PERFORMANCE:\n"
        for month, row in monthly_avg.sort_values('revenue', ascending=False).iterrows():
            month_name = month_names[int(month)-1]
            financial_doc += f"• {month_name}: ${row['revenue']:,.2f} revenue, {row['order_count']:.0f} orders, {row['active_customers']:.0f} customers\n"
        
        # Discount analysis
        financial_doc += f"\nDISCOUNT STRATEGY ANALYSIS:\n"
        financial_doc += f"• Average Discount Rate: {df_financial['avg_discount_percentage'].mean():.1f}%\n"
        financial_doc += f"• Total Discount Impact: ${total_discounts:,.2f} ({(total_discounts/(total_revenue+total_discounts)*100):.1f}% of gross revenue)\n"
        financial_doc += f"• Revenue Recovery Ratio: {((total_revenue/total_discounts) if total_discounts > 0 else 0):.1f}:1\n"
        
        documents.append(financial_doc)
        
        # 8. ADVANCED BUSINESS INTELLIGENCE AND INSIGHTS
        print("Generating advanced business intelligence insights...")
        
        # Customer loyalty and retention analysis
        loyalty_analysis_query = f"""
WITH customer_behavior AS (
    SELECT 
        c.customer_id,
        c.company_name,
        c.country,
        COUNT(DISTINCT o.order_id) as total_orders,
        COUNT(DISTINCT EXTRACT(year FROM o.order_date::date)) as years_active,
        COUNT(DISTINCT EXTRACT(month FROM o.order_date::date)) as months_active,
        MIN(o.order_date::date) as first_order,
        MAX(o.order_date::date) as last_order,
        SUM(od.quantity) as total_items,
        COUNT(DISTINCT od.product_id) as product_variety,
        ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as total_spent,
        -- Calculate customer lifetime in days instead
        MAX(o.order_date::date) - MIN(o.order_date::date) as customer_lifetime_days
    FROM northwind.customers c
    JOIN northwind.orders o ON c.customer_id = o.customer_id
    JOIN northwind.order_details od ON o.order_id = od.order_id
    WHERE o.order_date IS NOT NULL
    GROUP BY c.customer_id, c.company_name, c.country
)
SELECT *,
    CASE 
        WHEN total_orders >= 10 AND years_active >= 2 THEN 'Loyal'
        WHEN total_orders >= 5 AND years_active >= 1 THEN 'Regular'
        WHEN total_orders >= 3 THEN 'Developing'
        ELSE 'New'
    END as customer_segment,
    -- Calculate average days between orders as total lifetime / (orders - 1)
    CASE 
        WHEN total_orders > 1 THEN ROUND((customer_lifetime_days::numeric / (total_orders - 1)), 1)
        ELSE NULL
    END as avg_days_between_orders
FROM customer_behavior
ORDER BY total_spent DESC;
        """
        
        df_loyalty = pd.read_sql_query(loyalty_analysis_query, conn_string)
        
        # Product affinity analysis
        affinity_query = f"""
        WITH product_pairs AS (
            SELECT 
                od1.product_id as product_a,
                od2.product_id as product_b,
                COUNT(*) as co_occurrence
            FROM {schema}.order_details od1
            JOIN {schema}.order_details od2 ON od1.order_id = od2.order_id
            WHERE od1.product_id < od2.product_id
            GROUP BY od1.product_id, od2.product_id
            HAVING COUNT(*) >= 3
        )
        SELECT 
            pp.product_a,
            p1.product_name as product_a_name,
            pp.product_b,
            p2.product_name as product_b_name,
            pp.co_occurrence
        FROM product_pairs pp
        JOIN {schema}.products p1 ON pp.product_a = p1.product_id
        JOIN {schema}.products p2 ON pp.product_b = p2.product_id
        ORDER BY pp.co_occurrence DESC
        LIMIT 20
        """
        
        df_affinity = pd.read_sql_query(affinity_query, conn_string)
        
        insights_doc = "NORTHWIND ADVANCED BUSINESS INTELLIGENCE AND STRATEGIC INSIGHTS:\n\n"
        
        # Customer segmentation analysis
        segment_analysis = df_loyalty['customer_segment'].value_counts()
        
        insights_doc += "CUSTOMER LOYALTY SEGMENTATION:\n"
        for segment, count in segment_analysis.items():
            segment_customers = df_loyalty[df_loyalty['customer_segment'] == segment]
            avg_spend = segment_customers['total_spent'].mean()
            avg_orders = segment_customers['total_orders'].mean()
            
            insights_doc += f"• {segment} Customers: {count} ({(count/len(df_loyalty)*100):.1f}%)\n"
            insights_doc += f"  - Average Spend: ${avg_spend:,.2f}\n"
            insights_doc += f"  - Average Orders: {avg_orders:.1f}\n"
            insights_doc += f"  - Example: {segment_customers.iloc[0]['company_name']}\n"
        
        # High-value customer analysis
        insights_doc += "\nHIGH-VALUE CUSTOMER PROFILE:\n"
        top_customers = df_loyalty.head(10)
        insights_doc += f"• Top 10 customers represent ${top_customers['total_spent'].sum():,.2f} ({(top_customers['total_spent'].sum()/df_loyalty['total_spent'].sum()*100):.1f}% of total revenue)\n"
        insights_doc += f"• Average order frequency: {top_customers['avg_days_between_orders'].mean():.0f} days between orders\n"
        insights_doc += f"• Product diversity: {top_customers['product_variety'].mean():.0f} different products per customer\n"
        
        insights_doc += "\nTOP 10 MOST VALUABLE CUSTOMERS:\n"
        for i, row in top_customers.iterrows():
            customer_tenure = f"{row['first_order']:.10} to {row['last_order']:.10}"
            insights_doc += f"{i+1}. {row['company_name']} ({row['country']})\n"
            insights_doc += f"   • Total Value: ${row['total_spent']:,.2f} over {row['total_orders']} orders\n"
            insights_doc += f"   • Tenure: {customer_tenure} ({row['years_active']} years)\n"
            insights_doc += f"   • Behavior: {row['product_variety']} different products, avg {row['avg_days_between_orders']:.0f} days between orders\n"
            insights_doc += f"   • Segment: {row['customer_segment']}\n\n"
        
        # Product affinity insights
        insights_doc += "PRODUCT AFFINITY ANALYSIS (Frequently Bought Together):\n"
        insights_doc += "Products commonly purchased together can inform cross-selling strategies:\n\n"
        
        for i, row in df_affinity.head(15).iterrows():
            insights_doc += f"• {row['product_a_name']} + {row['product_b_name']}\n"
            insights_doc += f"  Purchased together in {row['co_occurrence']} orders\n"
        
        # Business recommendations
        insights_doc += "\nSTRATEGIC BUSINESS RECOMMENDATIONS:\n"
        
        # Revenue concentration analysis
        top_20_pct_customers = len(df_loyalty) // 5
        top_20_revenue = df_loyalty.head(top_20_pct_customers)['total_spent'].sum()
        total_revenue_check = df_loyalty['total_spent'].sum()
        
        insights_doc += f"• Revenue Concentration: Top 20% of customers generate ${top_20_revenue:,.2f} ({(top_20_revenue/total_revenue_check*100):.1f}% of revenue)\n"
        insights_doc += f"• Customer Retention: Focus on {segment_analysis['Loyal']} loyal customers who drive consistent revenue\n"
        insights_doc += f"• Growth Opportunity: {segment_analysis['Developing']} developing customers show potential for increased engagement\n"
        
        # Seasonal insights
        peak_months = df_financial.groupby('month')['revenue'].mean().nlargest(3)
        insights_doc += f"• Seasonal Strategy: Peak sales months are {', '.join([month_names[int(m)-1] for m in peak_months.index])}\n"
        
        # Geographic insights
        country_performance = df_loyalty.groupby('country').agg({
            'total_spent': 'sum',
            'customer_id': 'count'
        }).sort_values('total_spent', ascending=False)
        
        top_country = country_performance.index[0]
        insights_doc += f"• Geographic Focus: {top_country} is the top market with ${country_performance.loc[top_country, 'total_spent']:,.2f} from {country_performance.loc[top_country, 'customer_id']} customers\n"
        
        documents.append(insights_doc)
        
        # 9. OPERATIONAL EFFICIENCY AND INVENTORY INSIGHTS
        print("Generating operational efficiency analysis...")
        
        operational_query = f"""
WITH order_processing AS (
    SELECT 
        o.order_id,
        o.customer_id,
        o.employee_id,
        o.order_date::date,
        o.required_date::date,
        o.shipped_date::date,
        (o.shipped_date::date - o.order_date::date) as processing_days,
        (o.required_date::date - o.order_date::date) as promised_delivery_days,
        CASE WHEN o.shipped_date::date > o.required_date::date THEN 1 ELSE 0 END as late_delivery,
        o.freight,
        o.ship_country,
        COUNT(od.product_id) as items_in_order,
        SUM(od.quantity) as total_quantity,
        ROUND(SUM(od.unit_price * od.quantity * (1 - od.discount))::numeric, 2) as order_value
    FROM {schema}.orders o
    JOIN {schema}.order_details od ON o.order_id = od.order_id
    WHERE o.shipped_date IS NOT NULL AND o.order_date IS NOT NULL
    GROUP BY o.order_id, o.customer_id, o.employee_id, o.order_date, o.required_date, o.shipped_date, o.freight, o.ship_country
),
inventory_turnover AS (
    SELECT 
        p.product_id,
        p.product_name,
        c.category_name,
        p.units_in_stock,
        p.units_on_order,
        p.reorder_level,
        COALESCE(SUM(od.quantity), 0) as total_sold,
        CASE 
            WHEN p.units_in_stock > 0 THEN ROUND((COALESCE(SUM(od.quantity), 0)::numeric / p.units_in_stock), 2)
            ELSE 0 
        END as turnover_ratio
    FROM {schema}.products p
    JOIN {schema}.categories c ON p.category_id = c.category_id
    LEFT JOIN {schema}.order_details od ON p.product_id = od.product_id
    WHERE p.discontinued = 0
    GROUP BY p.product_id, p.product_name, c.category_name, p.units_in_stock, p.units_on_order, p.reorder_level
)
SELECT 
    'processing' as analysis_type,
    ROUND(AVG(processing_days)::numeric, 1) as avg_processing_days,
    ROUND(AVG(promised_delivery_days)::numeric, 1) as avg_promised_days,
    SUM(late_delivery) as total_late_deliveries,
    COUNT(*) as total_orders,
    ROUND(AVG(freight)::numeric, 2) as avg_freight_cost
FROM order_processing

UNION ALL

SELECT 
    'inventory' as analysis_type,
    ROUND(AVG(turnover_ratio)::numeric, 2) as avg_turnover,
    COUNT(CASE WHEN units_in_stock <= reorder_level THEN 1 END)::numeric as low_stock_items,
    COUNT(*)::numeric as total_active_products,
    SUM(units_in_stock)::numeric as total_inventory_units,
    ROUND(AVG(units_in_stock)::numeric, 1) as avg_stock_per_product
FROM inventory_turnover; """

        df_operational = pd.read_sql_query(operational_query, conn_string)
        
        
        # Detailed inventory analysis
        inventory_detail_query = f"""
        SELECT 
            p.product_name,
            c.category_name,
            p.units_in_stock,
            p.reorder_level,
            p.units_on_order,
            COALESCE(SUM(od.quantity), 0) as units_sold,
            CASE 
                WHEN p.units_in_stock > 0 AND COALESCE(SUM(od.quantity), 0) > 0
                THEN ROUND((COALESCE(SUM(od.quantity), 0) / p.units_in_stock)::numeric, 2)
                ELSE 0 
            END as turnover_ratio,
            CASE 
                WHEN p.units_in_stock <= p.reorder_level THEN 'LOW STOCK'
                WHEN p.units_in_stock = 0 THEN 'OUT OF STOCK'
                WHEN p.units_in_stock > p.reorder_level * 3 THEN 'OVERSTOCK'
                ELSE 'NORMAL'
            END as stock_status
        FROM {schema}.products p
        JOIN {schema}.categories c ON p.category_id = c.category_id
        LEFT JOIN {schema}.order_details od ON p.product_id = od.product_id
        WHERE p.discontinued = 0
        GROUP BY p.product_id, p.product_name, c.category_name, 
                 p.units_in_stock, p.reorder_level, p.units_on_order
        ORDER BY turnover_ratio DESC
        """
        
        df_inventory_detail = pd.read_sql_query(inventory_detail_query, conn_string)
        
        operational_doc = "NORTHWIND OPERATIONAL EFFICIENCY AND INVENTORY MANAGEMENT ANALYSIS:\n\n"
        
        # Extract operational metrics
        processing_metrics = df_operational[df_operational['analysis_type'] == 'processing'].iloc[0]
        inventory_metrics = df_operational[df_operational['analysis_type'] == 'inventory'].iloc[0]

        operational_doc += "ORDER PROCESSING EFFICIENCY:\n"
        operational_doc += f"• Average Processing Time: {processing_metrics['avg_processing_days']:.1f} days\n"
        operational_doc += f"• Average Promised Delivery: {processing_metrics['avg_promised_days']:.1f} days\n"
        operational_doc += f"• Late Deliveries: {processing_metrics['total_late_deliveries']:.0f} out of {processing_metrics['total_orders']:.0f} orders\n"
        operational_doc += f"• Average Freight Cost: ${processing_metrics['avg_freight_cost']:.2f} per shipment\n\n"

        operational_doc += "INVENTORY MANAGEMENT PERFORMANCE:\n"
        operational_doc += f"• Average Inventory Turnover Ratio: {inventory_metrics['avg_turnover']:.2f}\n"
        operational_doc += f"• Products Below Reorder Level: {inventory_metrics['low_stock_items']:.0f} out of {inventory_metrics['total_active_products']:.0f}\n"
        operational_doc += f"• Total Inventory Units: {inventory_metrics['total_inventory_units']:.0f}\n"
        operational_doc += f"• Average Stock per Product: {inventory_metrics['avg_stock_per_product']:.0f} units\n\n"        
        # Stock status analysis
        stock_status_summary = df_inventory_detail['stock_status'].value_counts()
        
        operational_doc += "INVENTORY STATUS BREAKDOWN:\n"
        for status, count in stock_status_summary.items():
            operational_doc += f"• {status}: {count} products ({(count/len(df_inventory_detail)*100):.1f}%)\n"
        
        # Critical inventory alerts
        critical_items = df_inventory_detail[df_inventory_detail['stock_status'].isin(['LOW STOCK', 'OUT OF STOCK'])]
        high_turnover_critical = critical_items[critical_items['turnover_ratio'] > inventory_metrics['avg_turnover']]
        
        operational_doc += f"\nCRITICAL INVENTORY ALERTS:\n"
        operational_doc += f"• Items Needing Immediate Attention: {len(critical_items)}\n"
        operational_doc += f"• High-Demand Items with Low Stock: {len(high_turnover_critical)}\n"
        
        if len(high_turnover_critical) > 0:
            operational_doc += "\nURGENT REORDER RECOMMENDATIONS:\n"
            for _, item in high_turnover_critical.head(10).iterrows():
                operational_doc += f"• {item['product_name']} ({item['category_name']})\n"
                operational_doc += f"  - Current Stock: {item['units_in_stock']}, Reorder Level: {item['reorder_level']}\n"
                operational_doc += f"  - Turnover Ratio: {item['turnover_ratio']:.2f}, On Order: {item['units_on_order']}\n"
        
        # Best performing inventory
        operational_doc += "\nTOP PERFORMING PRODUCTS (by turnover):\n"
        top_performers = df_inventory_detail[df_inventory_detail['turnover_ratio'] > 0].head(15)
        for _, item in top_performers.iterrows():
            operational_doc += f"• {item['product_name']}: {item['turnover_ratio']:.2f} turnover ratio\n"
            operational_doc += f"  - Stock: {item['units_in_stock']}, Sold: {item['units_sold']}, Status: {item['stock_status']}\n"
        
        documents.append(operational_doc)
        
        print(f"Successfully created {len(documents)} comprehensive business documents from Northwind PostgreSQL database")
        print(f"Total document length: {sum(len(doc) for doc in documents):,} characters")
        
        return documents
        
    except Exception as e:
        print(f"Error creating comprehensive documents: {e}")
        import traceback
        traceback.print_exc()
        return []

# # Usage example:
# if __name__ == "__main__":
#     # Neon PostgreSQL connection parameters
#     business_docs = create_comprehensive_northwind_business_documents(
#         host="ep-xxx-xxx.us-east-1.aws.neon.tech",  # Your Neon host
#         username="your_username",
#         password="your_password",
#         database="neondb",
#         port=5432,
#         schema="northwind"
#     )
    
#     # Preview the documents
#     for i, doc in enumerate(business_docs):
#         print(f"\n{'='*80}")
#         print(f"DOCUMENT {i+1}: {['CUSTOMER ANALYSIS', 'CUSTOMER BEHAVIOR', 'PRODUCT CATALOG', 'SUPPLIER ANALYSIS', 'EMPLOYEE PERFORMANCE', 'SHIPPING LOGISTICS', 'FINANCIAL PERFORMANCE', 'BUSINESS INTELLIGENCE', 'OPERATIONAL EFFICIENCY'][i]}")
#         print('='*80)
#         print(doc[:1000] + "..." if len(doc) > 1000 else doc)
    
    # Integration with vector store
    """
    from langchain.schema import Document
    
    # Convert to LangChain documents with detailed metadata
    langchain_docs = []
    doc_types = [
        "customer_analysis", "customer_behavior", "product_catalog", 
        "supplier_analysis", "employee_performance", "shipping_logistics",
        "financial_performance", "business_intelligence", "operational_efficiency"
    ]
    
    for i, doc in enumerate(business_docs):
        langchain_docs.append(Document(
            page_content=doc,
            metadata={
                "source": f"northwind_comprehensive_{doc_types[i]}",
                "type": "business_analysis",
                "document_id": i,
                "comprehensive": True,
                "data_source": "postgresql_neon"
            }
        ))
    
    # Enhanced text splitting for comprehensive documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,  # Larger chunks for comprehensive content
        chunk_overlap=100,  # More overlap for context preservation
        length_function=tiktoken_len,
    )
    
    split_chunks = text_splitter.split_documents(langchain_docs)
    
    # Create vector store
    qdrant_vectorstore = Qdrant.from_documents(
        split_chunks,
        embedding_model,
        location=":memory:",
        collection_name="northwind_comprehensive_business_data",
    )
    """

In [108]:
# # Required parameters only
# business_docs = create_northwind_business_documents(
#     host="your-neon-host.aws.neon.tech",
#     username="your_username", 
#     password="your_password"
# )

# # Or with custom database/schema
business_docs = create_comprehensive_northwind_business_documents(
    host="ep-aged-leaf-a5sdyft6-pooler.us-east-2.aws.neon.tech",
    username="neondb_owner",
    password="npg_m5bUF7retyMH",
    database="neondb",
    schema="northwind"
)

Connecting to Northwind database and generating comprehensive business documents...
Generating customer analysis document...
Generating customer purchasing behavior analysis...
Generating comprehensive product analysis...
Generating supplier analysis...
Generating employee and territory analysis...
Generating shipping and logistics analysis...
Generating comprehensive financial analysis...
Generating advanced business intelligence insights...
Generating operational efficiency analysis...
Error creating comprehensive documents: 'avg_turnover'


Traceback (most recent call last):
  File "/Users/viveknatan/Documents/AIMCourse/AIDA/New_Version_20250531/.venv/lib/python3.13/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'avg_turnover'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/9t/0z36p5hx6dd513bp6v3wxsz80000gn/T/ipykernel_54524/3253639268.py", line 865, in create_comprehensive_northwind_business_documents
    operational_doc += f"• Average Inventory Turnover Ratio: {i

In [ ]:
rag_chain_summary.invoke({"question" : "Who are the top customers of Northwind in the US?"})

'The provided context describes the Northwind sample database, which contains 14 interrelated tables relevant to a fictitious trading business, including data on orders, customers, and products. Specifically, the Orders table captures sales orders placed by customers, indicating a potential source for identifying customer activity and sales. However, the context does not include specific data or metrics regarding the top customers in the US or any sales figures that could be leveraged to ascertain customer rankings. Additionally, it references the CustomerDemographics table without providing details about its content. Overall, while the Northwind database features comprehensive customer and order information that could lead to identifying top customers, the exact details of those customers are not presented in the context provided.'

In [ ]:
rag_chain_summary.invoke({"question" : "Who are the top customers of Northwind in the US?"})

'The provided context describes the Northwind sample database, which contains 14 interrelated tables relevant to a fictitious trading business, including data on orders, customers, and products. Specifically, the Orders table captures sales orders placed by customers, indicating a potential source for identifying customer activity and sales. However, the context does not include specific data or metrics regarding the top customers in the US or any sales figures that could be leveraged to ascertain customer rankings. Additionally, it references the CustomerDemographics table without providing details about its content. Overall, while the Northwind database features comprehensive customer and order information that could lead to identifying top customers, the exact details of those customers are not presented in the context provided.'

In [ ]:
rag_chain_summary.invoke({"question" : "Who are the top customers of Northwind in the US?"})

'The provided context describes the Northwind sample database, which contains 14 interrelated tables relevant to a fictitious trading business, including data on orders, customers, and products. Specifically, the Orders table captures sales orders placed by customers, indicating a potential source for identifying customer activity and sales. However, the context does not include specific data or metrics regarding the top customers in the US or any sales figures that could be leveraged to ascertain customer rankings. Additionally, it references the CustomerDemographics table without providing details about its content. Overall, while the Northwind database features comprehensive customer and order information that could lead to identifying top customers, the exact details of those customers are not presented in the context provided.'

In [101]:
business_docs

[]